In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch
import torchvision
from PIL import Image
from torchvision import transforms
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as img
import torch.optim as optim
import os
import time
from torchvision.transforms import Resize, ToTensor, Normalize
import copy

In [2]:
DEVICE = torch.device('cuda')

In [3]:
df = pd.read_csv('label.csv')

In [80]:
class TRAIN_CDataset(Dataset):
    def __init__(self,train=True):
        self.df = pd.read_csv('./train_label.csv')
        #self.X = Image.open('./images/' + self.df.loc[idx]['path'] + '/' + self.df.loc[idx]['fname'])
        #self.y = self.df.loc[idx]['label']
        #self.d_idx = d_index
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        #data_index = self.d_idx
        X = Image.open('./images/' + self.df.loc[idx]['path'] + '/' + self.df.loc[idx]['fname'])
        X = transforms.CenterCrop(270)(X)
        y = self.df.loc[idx]['label'] 
        #print(y)
        X = transforms.ToTensor()(X)
        y = torch.tensor(y)
        return X,y

In [81]:
class TEST_CDataset(Dataset):
    def __init__(self,train=True):
        self.df = pd.read_csv('./test_label.csv')
        #self.X = Image.open('./images/' + self.df.loc[idx]['path'] + '/' + self.df.loc[idx]['fname'])
        #self.y = self.df.loc[idx]['label']
        #self.d_idx = d_index
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        #data_index = self.d_idx
        X = Image.open('./images/' + self.df.loc[idx]['path'] + '/' + self.df.loc[idx]['fname'])
        X = transforms.CenterCrop(270)(X)
        y = self.df.loc[idx]['label'] 
        #print(y)
        X = transforms.ToTensor()(X)
        y = torch.tensor(y)
        return X,y

In [82]:
train_ds = TRAIN_CDataset(train=True)
test_ds = TEST_CDataset(train=False)

In [83]:
train_dl = DataLoader(train_ds,
                      batch_size=16,
                      shuffle=True,
                      num_workers=1,
                     drop_last=True)
test_dl = DataLoader(test_ds,
                     batch_size=16,
                     shuffle=True,
                     num_workers=1,
                    drop_last=True)

In [65]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.residual_function = nn.Sequential(nn.Conv2d(in_channels, out_channels,kernel_size=3,
                                                         stride=stride, padding=1, bias=False),
                                               nn.BatchNorm2d(out_channels),
                                               nn.ReLU(),
                                               nn.Conv2d(out_channels, out_channels * BasicBlock.expansion,
                                                         kernel_size=3, stride=1, padding=1, bias=False),
                                               nn.BatchNorm2d(out_channels * BasicBlock.expansion),)
        self.shortcut = nn.Sequential()
        self.relu = nn.ReLU()
        
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(nn.Conv2d(in_channels, out_channels * BasicBlock.expansion,
                                                    kernel_size=1, stride=stride, bias=False),
                                          nn.BatchNorm2d(out_channels * BasicBlock.expansion))
    def forward(self,x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x

In [66]:
class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.residual_function = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=1,
                                                         stride=1,bias=False),
                                               nn.BatchNorm2d(out_channels),
                                               nn.ReLU(),
                                               nn.Conv2d(out_channels, out_channels, kernel_size=3,
                                                         stride=stride, padding=1, bias=False),
                                               nn.BatchNorm2d(out_channels),
                                               nn.ReLU(),
                                               nn.Conv2d(out_channels, out_channels * BottleNeck.expansion,
                                                         kernel_size=1, stride=1, bias=False),
                                               nn.BatchNorm2d(out_channels * BottleNeck.expansion),)
        self.shortcut = nn.Sequential()
        self.relu = nn.ReLU()
        
        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(nn.Conv2d(in_channels, out_channels * BottleNeck.expansion,
                                                     kernel_size=1, stride=stride, bias=False),
                                           nn.BatchNorm2d(out_channels * BottleNeck.expansion))
            
    def forward(self,x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        
        return x

In [67]:
class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=18, init_weights=True):
        super().__init__()

        self.in_channels=64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # weights inittialization
        if init_weights:
            self._initialize_weights()

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self,x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        x = self.conv3_x(output)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    # define weight initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [68]:
def resnet18():
    return ResNet(BasicBlock, [2,2,2,2])
def resnet34():
    return ResNet(BasicBlock, [3,4,6,3])
def resnet50():
    return ResNet(BottleNeck, [3,4,6,3])
def resnet101():
    return ResNet(BottleNeck, [3,4,23,3])
def resnet152():
    return ResNet(BottleNeck, [3,8,36,3])

In [69]:
device = torch.device('cuda')

In [70]:
#model = resnet50().to(device)
#x = torch.randn(3,3,244,244).to(device)
#output = model(x)
#print(output.size())

In [71]:
loss_func = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=0.001)

from torch.optim.lr_scheduler import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=10)

In [72]:
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']

In [73]:
def metric_batch(output, target):
    pred = output.argmax(1, keepdim=True)
    corrects = pred.eq(target.view_as(pred)).sum().item()
    return corrects

def loss_batch(loss_func, output, target, opt=None):
    loss = loss_func(output, target)
    metric_b = metric_batch(output, target)
    
    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()
        
    return loss.item(), metric_b

In [74]:
def loss_epoch(model, loss_func, dataset_dl, sanity_check=False, opt=None):
    running_loss = 0.0
    running_metric = 0.0
    len_data = len(dataset_dl.dataset)

    for xb, yb in dataset_dl:
        xb = xb.to(device)

        yb = yb.to(device)

        output = model(xb)


        loss_b, metric_b = loss_batch(loss_func, output, yb, opt)

        running_loss += loss_b
        
        if metric_b is not None:
            running_metric += metric_b
        
        if sanity_check is True:
            break

    loss = running_loss / len_data
    metric = running_metric / len_data

    return loss, metric

In [75]:
def train_val(model, params):
    num_epochs=params['num_epochs']
    loss_func=params["loss_func"]
    opt=params["optimizer"]
    train_dl=params["train_dl"]
    test_dl=params["test_dl"]
    sanity_check=params["sanity_check"]
    lr_scheduler=params["lr_scheduler"]
    path2weights=params["path2weights"]

    loss_history = {'train': [], 'val': []}
    metric_history = {'train': [], 'val': []}

    # # GPU out of memoty error
    #print(model.state_dict())
    #best_model_wts = copy.deepcopy(model.state_dict())

    best_loss = float('inf')

    start_time = time.time()

    for epoch in range(num_epochs):
        current_lr = get_lr(opt)
        print('Epoch {}/{}, current lr={}'.format(epoch, num_epochs-1, current_lr))

        model.train()
        train_loss, train_metric = loss_epoch(model, loss_func, train_dl, sanity_check, opt)
        loss_history['train'].append(train_loss)
        metric_history['train'].append(train_metric)

        model.eval()
        with torch.no_grad():
            val_loss, val_metric = loss_epoch(model, loss_func, test_dl, sanity_check)
        loss_history['val'].append(val_loss)
        metric_history['val'].append(val_metric)

        if val_loss < best_loss:
            best_loss = val_loss
            #print(model.state_dict())
            best_model_wts = copy.deepcopy(model.state_dict())
            #best_model_wts = model.state_dict()

            #torch.save(model.state_dict(), path2weights)
            #print('Copied best model weights!')
            print('Get best val_loss')

        lr_scheduler.step(val_loss)

        print('train loss: %.6f, val loss: %.6f, accuracy: %.2f, time: %.4f min' %(train_loss, val_loss, 100*val_metric, (time.time()-start_time)/60))
        print('-'*10)

    model.load_state_dict(best_model_wts)

    return model, loss_history, metric_history

In [76]:
# definc the training parameters
params_train = {
    'num_epochs':15,
    'optimizer':opt,
    'loss_func':loss_func,
    'train_dl':train_dl,
    'test_dl':test_dl,
    'sanity_check':False,
    'lr_scheduler':lr_scheduler,
    'path2weights':'./weights.pt',
}

# create the directory that stores weights.pt
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSerror:
        print('Error')
createFolder('./models')

In [84]:
test_dir = '/opt/ml/input/data/eval'
class Submission_Dataset(Dataset):
    def __init__(self,subm_path):
        self.subm_path = subm_path
        self.df = pd.read_csv(os.path.join(self.subm_path,'info.csv'))
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        X = Image.open(self.subm_path + '/images/' + self.df.loc[idx]['ImageID'])
        X = transforms.CenterCrop(270)(X)
        X = transforms.ToTensor()(X)
        return X

In [85]:
ex_subm_dataset = Submission_Dataset(test_dir)
ex_subm_dl = DataLoader(ex_subm_dataset,
                        shuffle=False)
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

In [86]:
model, loss_hist, metric_hist = train_val(model, params_train)

print('***'*10)
print('*#*'*10)
print('***'*10)

all_predictions = []
for images in ex_subm_dl:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'new_submission05.csv'), index=False)
print('test inference is done!')

Epoch 0/14, current lr=0.0001
Get best val_loss
train loss: 0.335828, val loss: 0.674461, accuracy: 76.43, time: 1.9763 min
----------
Epoch 1/14, current lr=0.0001
train loss: 0.269825, val loss: 0.732550, accuracy: 75.73, time: 3.9638 min
----------
Epoch 2/14, current lr=0.0001
train loss: 0.216128, val loss: 0.757877, accuracy: 74.76, time: 5.9545 min
----------
Epoch 3/14, current lr=0.0001
train loss: 0.181987, val loss: 0.908702, accuracy: 74.02, time: 7.9501 min
----------
Epoch 4/14, current lr=0.0001
train loss: 0.146163, val loss: 0.939734, accuracy: 74.19, time: 9.9280 min
----------
Epoch 5/14, current lr=0.0001
train loss: 0.119904, val loss: 1.051464, accuracy: 73.49, time: 11.9092 min
----------
Epoch 6/14, current lr=0.0001
train loss: 0.103494, val loss: 1.014809, accuracy: 74.54, time: 13.9105 min
----------
Epoch 7/14, current lr=0.0001
train loss: 0.077259, val loss: 1.139268, accuracy: 73.33, time: 15.9011 min
----------
Epoch 8/14, current lr=0.0001
train loss: 0

In [35]:
device

device(type='cuda')